### Exp: 2 Cluster customers first... Then map each cluster to vehicle

Assumption: Customer comes in cluster big small medium with some noise(somebody crossing da road) clustering them might mean they comes from the same vehicle (or come together from somewhere)

Cross : Create a HDBSCAN cluster then tune it accordingly

Cross OUTPUT : 
- df_cross['cluster'] : the cluster that the that customer belongs to with 
- cluster_cross : the cluster info of cross containin 
    - xy mid mean, min , max - timestamp (start stop avg) and COUNT of cluster size


camera used : mbk-14-12-vehicle 

### Load

In [1]:
import sys

sys.path.append("/home/jeans/internship/parking-customer-count")

from countpassenger.Config import conf
from countpassenger import Preprocess
from countpassenger import Cluster

import countpassenger
import pandas as pd
import numpy as np

import os.path as osp
import os

CURRENT_CAM = "mbk-14-12"

df_vehicle = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-vehicle-object-20240428.csv")
)
df_cross = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-cross-object-20240428.csv")
)
df_reverse = pd.read_csv(
    osp.join(conf.RESOURCES_RAW_DIR, "2024-04-28/mbk-tourist-raw-reverse-object-20240428.csv")
)

df_vehicle = Preprocess.df_clean_vehicle(
    df_vehicle=df_vehicle,
)
df_vehicle = Preprocess.filter_camera(df_vehicle, camera_name=CURRENT_CAM + "-vehicle")
df_vehicle = Preprocess.sort_df(df_vehicle, conf.VEHICLE_INTEREST_NON_SNAPSHOT)
print(df_vehicle.to_string())
# print(df_vehicle[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT]].to_string())

      is_plate_valid  travel_distance  vehicle_type_confidence  lifetime  ymin plate_number  xmin  province_confidence similarized_plate_number  plate_number_confidence  xmax                                                                                      image_name purchasing_power province             camera  travel_displacement            timestamp   car_brand  travel_trajectory  frame_no               original_video_name  ymax  plate_color_confidence vehicle_type  car_brand_confidence plate_color  plate_image_name       timestamp_precise           true_video_name                        video_name    xmid   ymid  timestamp_unix  timestamp_unix_end
2315            True      2828.286892                 0.355878    14.236     0       327722  1221             0.951550                   327722                 0.993560  2191   2024-04-28T09-27-41_FLAG_P_FID_1291_OID_53g0_mbk-14-12-vehicle_1714271101_1714271261_1291.jpg          economy    th-10  mbk-14-12-vehicle           978.357679 

### Preprocess

In [2]:
df_cross = Preprocess.df_clean_customer(df_cross)
df_cross = Preprocess.filter_camera(df_cross, camera_name=CURRENT_CAM)
df_cross = Preprocess.sort_df(df_cross, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT,'timestamp_unix']].to_string())

In [3]:
df_reverse = Preprocess.df_clean_customer(df_reverse)
df_reverse = Preprocess.filter_camera(df_reverse, camera_name=CURRENT_CAM)
df_reverse = Preprocess.sort_df(df_reverse, conf.CUSTOMER_INTEREST_SNAPSHOT)

# print(df_reverse[[*conf.CUSTOMER_INTEREST_SNAPSHOT, "timestamp_unix"]].to_string())

### Clustering the customer

In [4]:
df_cross ,  cluster_cross = Cluster.perform_cross_clustering(df_cross=df_cross)

# print(df_cross[[*conf.CUSTOMER_INTEREST_SNAPSHOT, 'cluster']].to_string())

In [5]:
cluster_cross

,cluster_id,xmid_min,xmid_max,xmid_mean,ymid_min,ymid_max,ymid_mean,timestamp_unix_min,timestamp_unix_max,timestamp_unix_mean,count
0,-1,280.5,2163.5,1650.337500,32.5,909.5,305.862500,1.714270e+09,1.714318e+09,1.714296e+09,40
1,0,547.5,2126.0,1336.750000,46.5,813.5,430.000000,1.714270e+09,1.714270e+09,1.714270e+09,2
2,1,1490.0,2127.5,1876.333333,40.5,585.0,261.833333,1.714272e+09,1.714273e+09,1.714272e+09,3
3,2,270.0,2124.5,967.666667,91.0,781.5,540.500000,1.714274e+09,1.714274e+09,1.714274e+09,3
4,3,1275.0,2120.0,1837.666667,79.5,565.0,242.000000,1.714281e+09,1.714282e+09,1.714281e+09,3
...,...,...,...,...,...,...,...,...,...,...,...
149,148,2115.5,2124.0,2119.750000,73.5,83.0,78.250000,1.714295e+09,1.714295e+09,1.714295e+09,2
150,149,1954.0,2108.5,2049.833333,115.5,190.5,144.833333,1.714297e+09,1.714297e+09,1.714297e+09,3
151,150,2126.5,2126.5,2126.500000,56.0,56.0,56.000000,1.714297e+09,1.714297e+09,1.714297e+09,2
152,151,2117.5,2157.5,2129.958333,56.0,85.0,69.666667,1.714295e+09,1.714295e+09,1.714295e+09,12


In [7]:
john =  Cluster.assign_cluster_to_vehicle(df_vehicle, cluster_cross)
print(john[[*conf.VEHICLE_INTEREST_NON_SNAPSHOT_CLUSTERED]].to_string())

,camera,timestamp_precise,lifetime,xmid,ymid,plate_number,vehicle_type,timestamp_unix,timestamp_unix_end,cluster_list,count
2315,mbk-14-12-vehicle,2024-04-28 02:27:41.480,14.236,1706.0,194.0,327722,pickup,1714271261,1714271275,"[0.0, 13.0, 28.0, 29.0]",10
2242,mbk-14-12-vehicle,2024-04-28 03:08:09.313,18.355,1564.5,181.5,4กบ5887,pickup,1714273689,1714273707,"[1.0, 2.0, 5.0, 6.0, 14.0, 15.0]",32
2241,mbk-14-12-vehicle,2024-04-28 03:08:27.068,49.866,1397.0,140.5,6กถ9907,pickup,1714273707,1714273756,[7.0],4
2215,mbk-14-12-vehicle,2024-04-28 03:18:58.899,46.827,1551.5,172.0,อก5,pickup,1714274338,1714274384,[8.0],2
2204,mbk-14-12-vehicle,2024-04-28 03:21:31.980,2.639,1401.5,174.0,ทห1197,pickup,1714274491,1714274493,[],0
...,...,...,...,...,...,...,...,...,...,...,...
233,mbk-14-12-vehicle,2024-04-28 14:31:57.614,220.742,1373.5,127.5,1มค4027,taxi,1714314717,1714314937,[],0
234,mbk-14-12-vehicle,2024-04-28 14:41:14.864,7.598,1624.0,196.0,NaN,motorbike,1714315274,1714315281,[65.0],4
211,mbk-14-12-vehicle,2024-04-28 14:45:01.523,86.856,1364.0,122.5,1มก8802,taxi,1714315501,1714315587,[],0
152,mbk-14-12-vehicle,2024-04-28 15:09:09.816,26.473,1356.0,125.0,กพ9715,motorbike,1714316949,1714316975,"[66.0, 99.0, 100.0]",9


TODO: find a better metric function to calculate distances